In [1]:
import higlass as hg
import cooler
import os
from clodius.cli import aggregate

In [2]:
def compute_domain(coords, clr: cooler.Cooler, padding=2.0):
    df = clr.matrix(as_pixels=True).fetch(coords)

    pos1 = df["bin1_id"].min() * clr.binsize
    pos2 = df["bin1_id"].max() * clr.binsize

    if padding != 0.0:
        extent = pos2 - pos1
        new_extent = int(padding * extent)
        pos1 -= (new_extent - extent) // 2
        pos2 += (new_extent - extent) // 2

    return max(0, pos1), pos2


def parse_coords(coords):
    chrom, _, coords = coords.partition(":")
    start, _, end = coords.partition("-")
    start = int(start.replace(",", ""))
    end = int(end.replace(",", ""))

    return chrom, start, end


def expand_coords(coords, chroms, padding=1_000_000):
    chrom, start, end = parse_coords(coords)
    padding_per_side = padding / 2
    start = max(0, start - padding_per_side)
    end = min(chroms[chrom], end + padding_per_side)

    return f"{chrom}:{start:.0f}-{end:.0f}"

def prepare_tads(path_to_tads, outdir):
    name = os.path.basename(path_to_tads).removesuffix(".bed.gz")
    out_path = os.path.join(outdir, f"{name}.beddb")
    aggregate._bedpe(
        path_to_tads,
        out_path,
        assembly="hg38",
        chr1_col=1,
        from1_col=2,
        to1_col=3,
        chr2_col=1,
        from2_col=2,
        to2_col=3
    )
    return out_path

In [3]:
gene_annotation_tile = hg.remote(
    uid="P0PLbQMwTYGy-5uPIQid7A",
    server="https://higlass.io/api/v1/",
    name="hg38 Gene annotation",
)

coolers = {
    "WT": "../data/output/nfcore_hic/mcools/hg38_MCF10A_WT_merged.mcool",
    "T1": "../data/output/nfcore_hic/mcools/hg38_MCF10A_T1_merged.mcool",
    "C1": "../data/output/nfcore_hic/mcools/hg38_MCF10A_C1_merged.mcool"
}

tads = {
    "WT": ["../data/output/tad_analysis/ICE/50000/hg38_001_MCF10A_WT_REP1_domains.bed.gz",
           "../data/output/tad_analysis/ICE/50000/hg38_002_MCF10A_WT_REP2_domains.bed.gz"],
    "T1": ["../data/output/tad_analysis/ICE/50000/hg38_003_MCF10A_T1_REP1_domains.bed.gz",
           "../data/output/tad_analysis/ICE/50000/hg38_004_MCF10A_T1_REP2_domains.bed.gz"],
    "C1": ["../data/output/tad_analysis/ICE/50000/hg38_005_MCF10A_C1_REP1_domains.bed.gz",
           "../data/output/tad_analysis/ICE/50000/hg38_006_MCF10A_C1_REP2_domains.bed.gz"],
}

genes = {
    "BRCA1": "chr17:43,044,295-43,170,245",
    "BRCA2": "chr13:32,315,086-32,400,268",
    "TP53": "chr17:7,661,779-7,687,546",
    "PTEN": "chr10:87,862,638-87,971,930",
    "PIK3CA": "chr3:179,148,114-179,240,093",
    "ERBB2": "chr17:39,687,914-39,730,426",
    "MYC": "chr8:127,735,434-127,742,951"
}

tmpdir = "/tmp/visualizing_tads_with_bc_oncogenes"
os.makedirs(tmpdir, exist_ok=True)

domains_beddb = {}


chroms = cooler.Cooler(f"{list(coolers.values())[0]}::/resolutions/1000000").chroms()[:].set_index("name")["length"].to_dict()


cooler_tracks = {}
domains_tracks = {}
chrom_sizes_tracks = {}
hga_tracks = {}
vga_tracks = {}


for sample, (path1, path2) in tads.items():
    domains_tracks[sample] = [hg.bed2ddb(prepare_tads(path1, tmpdir)).track("bedlike", height=25)]
    domains_tracks[sample].append(hg.bed2ddb(prepare_tads(path2, tmpdir)).track("bedlike", height=25))


tracks = {}

for sample, clr in coolers.items():
    cooler_tracks[sample] = hg.cooler(clr).track("linear-heatmap", height=200, options={"labelPosition": "hidden"})
    
    chrom_sizes_tracks[sample] = hg.cooler(clr).track("chromosome-labels", height=25, options={"fontSize": 16})
    hga_tracks[sample] = gene_annotation_tile.track("horizontal-gene-annotations", height=150, options={"fontSize": 16})
    vga_tracks[sample] = gene_annotation_tile.track("vertical-gene-annotations", width=150)


uri = list(coolers.values())[0]
if cooler.fileops.is_multires_file(uri):
    grps = cooler.fileops.list_coolers(uri)
    uri = f"{uri}::{grps[0]}"

In [4]:
domain = compute_domain(expand_coords(genes["BRCA1"], chroms), cooler.Cooler(uri))

width = 6

view = hg.view(
    (hga_tracks["WT"], "top"),
    (chrom_sizes_tracks["WT"], "top"),
    (cooler_tracks["WT"], "top"),
    (domains_tracks["WT"][0], "top"),
    (domains_tracks["WT"][1], "top"),
    (cooler_tracks["T1"], "top"),
    (domains_tracks["T1"][0], "top"),
    (domains_tracks["T1"][1], "top"),
    (cooler_tracks["C1"], "top"),
    (domains_tracks["C1"][0], "top"),
    (domains_tracks["C1"][1], "top"),
    width=width).domain(x=domain, y=domain)

view

View(layout=Layout(x=0, y=0, w=6, h=6, moved=None, static=None), tracks=Tracks[TypeVar](left=None, right=None, top=[EnumTrack(tilesetUid='P0PLbQMwTYGy-5uPIQid7A', server='https://higlass.io/api/v1/', type='horizontal-gene-annotations', uid='bb38e62b-9445-485e-87ba-770e494a66bf', width=None, height=150, options={'fontSize': 16, 'name': 'hg38 Gene annotation'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='chromosome-labels', uid='d50b64d1-32f5-4eb0-85e9-c94f8b8b611c', width=None, height=25, options={'fontSize': 16}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='9acea9b7-63bc-41c1-8385-bdd7661038a8', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='5175a387dd4da2d13131fd51327f7d46', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='a22515de-1c8a-42be-af5f-0694a1bb15cd', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='e310bf79cbf002d5000473d81f4e5a5e', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='708d470a-513a-4040-afbd-a41919b64c37', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='c961bde973e0d73cc26446b7532988f9', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='a250b64e-0648-47ec-9aee-401ff7a0917d', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='593a0f866eb80cf9d5cca970fa417527', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='5afde906-5b23-4c86-9adb-1f45c0833dc2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='58a9523618128cb852d3c27df2cc37e0', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='36210369-09bf-4c2a-8f62-3fe2031c304e', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='02e969c381056765f1f31fcee7cd23be', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='71e4eab1-d0bf-48d2-9f8c-a41bbfa219c5', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='20201ab170dd1c91ffc3328ae4d6b306', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='7a56c061-c9ce-44c8-b1f1-1f0e613eaed2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='80f69328ecd359f8badc793360869617', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='2592e9b9-9535-4c49-b9e8-0a41b9c48d14', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=None, whole=None, gallery=None), uid='9554b63f-6649-4c5f-8b1b-8175aa3f48a1', autocompleteSource=None, chromInfoPath=None, genomePositionSearchBox=None, genomePositionSearchBoxVisible=None, initialXDomain=(2532769000.0, 2535021000.0), initialYDomain=(2532769000.0, 2535021000.0), overlays=None, selectionView=None, zoomFixed=None, zoomLimits=(1.0, None))

In [5]:
domain = compute_domain(expand_coords(genes["BRCA2"], chroms), cooler.Cooler(uri))

width = 6

view = hg.view(
    (hga_tracks["WT"], "top"),
    (chrom_sizes_tracks["WT"], "top"),
    (cooler_tracks["WT"], "top"),
    (domains_tracks["WT"][0], "top"),
    (domains_tracks["WT"][1], "top"),
    (cooler_tracks["T1"], "top"),
    (domains_tracks["T1"][0], "top"),
    (domains_tracks["T1"][1], "top"),
    (cooler_tracks["C1"], "top"),
    (domains_tracks["C1"][0], "top"),
    (domains_tracks["C1"][1], "top"),
    width=width).domain(x=domain, y=domain)

view

View(layout=Layout(x=0, y=0, w=6, h=6, moved=None, static=None), tracks=Tracks[TypeVar](left=None, right=None, top=[EnumTrack(tilesetUid='P0PLbQMwTYGy-5uPIQid7A', server='https://higlass.io/api/v1/', type='horizontal-gene-annotations', uid='bb38e62b-9445-485e-87ba-770e494a66bf', width=None, height=150, options={'fontSize': 16, 'name': 'hg38 Gene annotation'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='chromosome-labels', uid='d50b64d1-32f5-4eb0-85e9-c94f8b8b611c', width=None, height=25, options={'fontSize': 16}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='9acea9b7-63bc-41c1-8385-bdd7661038a8', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='5175a387dd4da2d13131fd51327f7d46', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='a22515de-1c8a-42be-af5f-0694a1bb15cd', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='e310bf79cbf002d5000473d81f4e5a5e', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='708d470a-513a-4040-afbd-a41919b64c37', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='c961bde973e0d73cc26446b7532988f9', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='a250b64e-0648-47ec-9aee-401ff7a0917d', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='593a0f866eb80cf9d5cca970fa417527', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='5afde906-5b23-4c86-9adb-1f45c0833dc2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='58a9523618128cb852d3c27df2cc37e0', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='36210369-09bf-4c2a-8f62-3fe2031c304e', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='02e969c381056765f1f31fcee7cd23be', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='71e4eab1-d0bf-48d2-9f8c-a41bbfa219c5', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='20201ab170dd1c91ffc3328ae4d6b306', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='7a56c061-c9ce-44c8-b1f1-1f0e613eaed2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='80f69328ecd359f8badc793360869617', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='2592e9b9-9535-4c49-b9e8-0a41b9c48d14', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=None, whole=None, gallery=None), uid='58fed266-6a6f-4843-867b-4b7eb9ec1cf6', autocompleteSource=None, chromInfoPath=None, genomePositionSearchBox=None, genomePositionSearchBoxVisible=None, initialXDomain=(2108320500.0, 2110490500.0), initialYDomain=(2108320500.0, 2110490500.0), overlays=None, selectionView=None, zoomFixed=None, zoomLimits=(1.0, None))

In [6]:
domain = compute_domain(expand_coords(genes["TP53"], chroms), cooler.Cooler(uri))

width = 6

view = hg.view(
    (hga_tracks["WT"], "top"),
    (chrom_sizes_tracks["WT"], "top"),
    (cooler_tracks["WT"], "top"),
    (domains_tracks["WT"][0], "top"),
    (domains_tracks["WT"][1], "top"),
    (cooler_tracks["T1"], "top"),
    (domains_tracks["T1"][0], "top"),
    (domains_tracks["T1"][1], "top"),
    (cooler_tracks["C1"], "top"),
    (domains_tracks["C1"][0], "top"),
    (domains_tracks["C1"][1], "top"),
    width=width).domain(x=domain, y=domain)

view

View(layout=Layout(x=0, y=0, w=6, h=6, moved=None, static=None), tracks=Tracks[TypeVar](left=None, right=None, top=[EnumTrack(tilesetUid='P0PLbQMwTYGy-5uPIQid7A', server='https://higlass.io/api/v1/', type='horizontal-gene-annotations', uid='bb38e62b-9445-485e-87ba-770e494a66bf', width=None, height=150, options={'fontSize': 16, 'name': 'hg38 Gene annotation'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='chromosome-labels', uid='d50b64d1-32f5-4eb0-85e9-c94f8b8b611c', width=None, height=25, options={'fontSize': 16}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='9acea9b7-63bc-41c1-8385-bdd7661038a8', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='5175a387dd4da2d13131fd51327f7d46', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='a22515de-1c8a-42be-af5f-0694a1bb15cd', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='e310bf79cbf002d5000473d81f4e5a5e', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='708d470a-513a-4040-afbd-a41919b64c37', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='c961bde973e0d73cc26446b7532988f9', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='a250b64e-0648-47ec-9aee-401ff7a0917d', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='593a0f866eb80cf9d5cca970fa417527', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='5afde906-5b23-4c86-9adb-1f45c0833dc2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='58a9523618128cb852d3c27df2cc37e0', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='36210369-09bf-4c2a-8f62-3fe2031c304e', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='02e969c381056765f1f31fcee7cd23be', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='71e4eab1-d0bf-48d2-9f8c-a41bbfa219c5', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='20201ab170dd1c91ffc3328ae4d6b306', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='7a56c061-c9ce-44c8-b1f1-1f0e613eaed2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='80f69328ecd359f8badc793360869617', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='2592e9b9-9535-4c49-b9e8-0a41b9c48d14', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=None, whole=None, gallery=None), uid='1578cb71-d878-48ff-a4af-9948066fd9b0', autocompleteSource=None, chromInfoPath=None, genomePositionSearchBox=None, genomePositionSearchBoxVisible=None, initialXDomain=(2497436000.0, 2499488000.0), initialYDomain=(2497436000.0, 2499488000.0), overlays=None, selectionView=None, zoomFixed=None, zoomLimits=(1.0, None))

In [7]:
domain = compute_domain(expand_coords(genes["PTEN"], chroms), cooler.Cooler(uri))

width = 6

view = hg.view(
    (hga_tracks["WT"], "top"),
    (chrom_sizes_tracks["WT"], "top"),
    (cooler_tracks["WT"], "top"),
    (domains_tracks["WT"][0], "top"),
    (domains_tracks["WT"][1], "top"),
    (cooler_tracks["T1"], "top"),
    (domains_tracks["T1"][0], "top"),
    (domains_tracks["T1"][1], "top"),
    (cooler_tracks["C1"], "top"),
    (domains_tracks["C1"][0], "top"),
    (domains_tracks["C1"][1], "top"),
    width=width).domain(x=domain, y=domain)

view

View(layout=Layout(x=0, y=0, w=6, h=6, moved=None, static=None), tracks=Tracks[TypeVar](left=None, right=None, top=[EnumTrack(tilesetUid='P0PLbQMwTYGy-5uPIQid7A', server='https://higlass.io/api/v1/', type='horizontal-gene-annotations', uid='bb38e62b-9445-485e-87ba-770e494a66bf', width=None, height=150, options={'fontSize': 16, 'name': 'hg38 Gene annotation'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='chromosome-labels', uid='d50b64d1-32f5-4eb0-85e9-c94f8b8b611c', width=None, height=25, options={'fontSize': 16}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='9acea9b7-63bc-41c1-8385-bdd7661038a8', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='5175a387dd4da2d13131fd51327f7d46', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='a22515de-1c8a-42be-af5f-0694a1bb15cd', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='e310bf79cbf002d5000473d81f4e5a5e', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='708d470a-513a-4040-afbd-a41919b64c37', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='c961bde973e0d73cc26446b7532988f9', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='a250b64e-0648-47ec-9aee-401ff7a0917d', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='593a0f866eb80cf9d5cca970fa417527', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='5afde906-5b23-4c86-9adb-1f45c0833dc2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='58a9523618128cb852d3c27df2cc37e0', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='36210369-09bf-4c2a-8f62-3fe2031c304e', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='02e969c381056765f1f31fcee7cd23be', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='71e4eab1-d0bf-48d2-9f8c-a41bbfa219c5', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='20201ab170dd1c91ffc3328ae4d6b306', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='7a56c061-c9ce-44c8-b1f1-1f0e613eaed2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='80f69328ecd359f8badc793360869617', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='2592e9b9-9535-4c49-b9e8-0a41b9c48d14', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=None, whole=None, gallery=None), uid='85c0b4c3-2450-41ec-9b39-acb270823956', autocompleteSource=None, chromInfoPath=None, genomePositionSearchBox=None, genomePositionSearchBoxVisible=None, initialXDomain=(1761694500.0, 1763912500.0), initialYDomain=(1761694500.0, 1763912500.0), overlays=None, selectionView=None, zoomFixed=None, zoomLimits=(1.0, None))

In [8]:
domain = compute_domain(expand_coords(genes["PIK3CA"], chroms), cooler.Cooler(uri))

width = 6

view = hg.view(
    (hga_tracks["WT"], "top"),
    (chrom_sizes_tracks["WT"], "top"),
    (cooler_tracks["WT"], "top"),
    (domains_tracks["WT"][0], "top"),
    (domains_tracks["WT"][1], "top"),
    (cooler_tracks["T1"], "top"),
    (domains_tracks["T1"][0], "top"),
    (domains_tracks["T1"][1], "top"),
    (cooler_tracks["C1"], "top"),
    (domains_tracks["C1"][0], "top"),
    (domains_tracks["C1"][1], "top"),
    width=width).domain(x=domain, y=domain)

view

View(layout=Layout(x=0, y=0, w=6, h=6, moved=None, static=None), tracks=Tracks[TypeVar](left=None, right=None, top=[EnumTrack(tilesetUid='P0PLbQMwTYGy-5uPIQid7A', server='https://higlass.io/api/v1/', type='horizontal-gene-annotations', uid='bb38e62b-9445-485e-87ba-770e494a66bf', width=None, height=150, options={'fontSize': 16, 'name': 'hg38 Gene annotation'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='chromosome-labels', uid='d50b64d1-32f5-4eb0-85e9-c94f8b8b611c', width=None, height=25, options={'fontSize': 16}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='9acea9b7-63bc-41c1-8385-bdd7661038a8', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='5175a387dd4da2d13131fd51327f7d46', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='a22515de-1c8a-42be-af5f-0694a1bb15cd', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='e310bf79cbf002d5000473d81f4e5a5e', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='708d470a-513a-4040-afbd-a41919b64c37', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='c961bde973e0d73cc26446b7532988f9', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='a250b64e-0648-47ec-9aee-401ff7a0917d', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='593a0f866eb80cf9d5cca970fa417527', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='5afde906-5b23-4c86-9adb-1f45c0833dc2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='58a9523618128cb852d3c27df2cc37e0', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='36210369-09bf-4c2a-8f62-3fe2031c304e', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='02e969c381056765f1f31fcee7cd23be', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='71e4eab1-d0bf-48d2-9f8c-a41bbfa219c5', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='20201ab170dd1c91ffc3328ae4d6b306', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='7a56c061-c9ce-44c8-b1f1-1f0e613eaed2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='80f69328ecd359f8badc793360869617', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='2592e9b9-9535-4c49-b9e8-0a41b9c48d14', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=None, whole=None, gallery=None), uid='1a78413d-a275-43dd-be4c-316a5afcfcbc', autocompleteSource=None, chromInfoPath=None, genomePositionSearchBox=None, genomePositionSearchBoxVisible=None, initialXDomain=(669253000.0, 671437000.0), initialYDomain=(669253000.0, 671437000.0), overlays=None, selectionView=None, zoomFixed=None, zoomLimits=(1.0, None))

In [9]:
domain = compute_domain(expand_coords(genes["ERBB2"], chroms), cooler.Cooler(uri))

width = 6

view = hg.view(
    (hga_tracks["WT"], "top"),
    (chrom_sizes_tracks["WT"], "top"),
    (cooler_tracks["WT"], "top"),
    (domains_tracks["WT"][0], "top"),
    (domains_tracks["WT"][1], "top"),
    (cooler_tracks["T1"], "top"),
    (domains_tracks["T1"][0], "top"),
    (domains_tracks["T1"][1], "top"),
    (cooler_tracks["C1"], "top"),
    (domains_tracks["C1"][0], "top"),
    (domains_tracks["C1"][1], "top"),
    width=width).domain(x=domain, y=domain)

view

View(layout=Layout(x=0, y=0, w=6, h=6, moved=None, static=None), tracks=Tracks[TypeVar](left=None, right=None, top=[EnumTrack(tilesetUid='P0PLbQMwTYGy-5uPIQid7A', server='https://higlass.io/api/v1/', type='horizontal-gene-annotations', uid='bb38e62b-9445-485e-87ba-770e494a66bf', width=None, height=150, options={'fontSize': 16, 'name': 'hg38 Gene annotation'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='chromosome-labels', uid='d50b64d1-32f5-4eb0-85e9-c94f8b8b611c', width=None, height=25, options={'fontSize': 16}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='9acea9b7-63bc-41c1-8385-bdd7661038a8', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='5175a387dd4da2d13131fd51327f7d46', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='a22515de-1c8a-42be-af5f-0694a1bb15cd', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='e310bf79cbf002d5000473d81f4e5a5e', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='708d470a-513a-4040-afbd-a41919b64c37', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='c961bde973e0d73cc26446b7532988f9', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='a250b64e-0648-47ec-9aee-401ff7a0917d', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='593a0f866eb80cf9d5cca970fa417527', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='5afde906-5b23-4c86-9adb-1f45c0833dc2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='58a9523618128cb852d3c27df2cc37e0', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='36210369-09bf-4c2a-8f62-3fe2031c304e', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='02e969c381056765f1f31fcee7cd23be', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='71e4eab1-d0bf-48d2-9f8c-a41bbfa219c5', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='20201ab170dd1c91ffc3328ae4d6b306', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='7a56c061-c9ce-44c8-b1f1-1f0e613eaed2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='80f69328ecd359f8badc793360869617', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='2592e9b9-9535-4c49-b9e8-0a41b9c48d14', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=None, whole=None, gallery=None), uid='90c41b1a-591e-41a4-85f0-d539400f5681', autocompleteSource=None, chromInfoPath=None, genomePositionSearchBox=None, genomePositionSearchBoxVisible=None, initialXDomain=(2529453500.0, 2531539500.0), initialYDomain=(2529453500.0, 2531539500.0), overlays=None, selectionView=None, zoomFixed=None, zoomLimits=(1.0, None))

In [10]:
domain = compute_domain(expand_coords(genes["MYC"], chroms), cooler.Cooler(uri))

width = 6

view = hg.view(
    (hga_tracks["WT"], "top"),
    (chrom_sizes_tracks["WT"], "top"),
    (cooler_tracks["WT"], "top"),
    (domains_tracks["WT"][0], "top"),
    (domains_tracks["WT"][1], "top"),
    (cooler_tracks["T1"], "top"),
    (domains_tracks["T1"][0], "top"),
    (domains_tracks["T1"][1], "top"),
    (cooler_tracks["C1"], "top"),
    (domains_tracks["C1"][0], "top"),
    (domains_tracks["C1"][1], "top"),
    width=width).domain(x=domain, y=domain)

view

View(layout=Layout(x=0, y=0, w=6, h=6, moved=None, static=None), tracks=Tracks[TypeVar](left=None, right=None, top=[EnumTrack(tilesetUid='P0PLbQMwTYGy-5uPIQid7A', server='https://higlass.io/api/v1/', type='horizontal-gene-annotations', uid='bb38e62b-9445-485e-87ba-770e494a66bf', width=None, height=150, options={'fontSize': 16, 'name': 'hg38 Gene annotation'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='chromosome-labels', uid='d50b64d1-32f5-4eb0-85e9-c94f8b8b611c', width=None, height=25, options={'fontSize': 16}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='47dfa20b5102f7edb44e596136656640', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='9acea9b7-63bc-41c1-8385-bdd7661038a8', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='5175a387dd4da2d13131fd51327f7d46', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='a22515de-1c8a-42be-af5f-0694a1bb15cd', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='e310bf79cbf002d5000473d81f4e5a5e', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='708d470a-513a-4040-afbd-a41919b64c37', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='c961bde973e0d73cc26446b7532988f9', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='a250b64e-0648-47ec-9aee-401ff7a0917d', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='593a0f866eb80cf9d5cca970fa417527', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='5afde906-5b23-4c86-9adb-1f45c0833dc2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='58a9523618128cb852d3c27df2cc37e0', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='36210369-09bf-4c2a-8f62-3fe2031c304e', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='02e969c381056765f1f31fcee7cd23be', server='http://localhost:40825/tilesets/api/v1/', type='linear-heatmap', uid='71e4eab1-d0bf-48d2-9f8c-a41bbfa219c5', width=None, height=200, options={'labelPosition': 'hidden'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='20201ab170dd1c91ffc3328ae4d6b306', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='7a56c061-c9ce-44c8-b1f1-1f0e613eaed2', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='80f69328ecd359f8badc793360869617', server='http://localhost:40825/tilesets/api/v1/', type='bedlike', uid='2592e9b9-9535-4c49-b9e8-0a41b9c48d14', width=None, height=25, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=None, whole=None, gallery=None), uid='a73776a7-c73e-440e-b260-1d627ce91d22', autocompleteSource=None, chromInfoPath=None, genomePositionSearchBox=None, genomePositionSearchBoxVisible=None, initialXDomain=(1518084500.0, 1520098500.0), initialYDomain=(1518084500.0, 1520098500.0), overlays=None, selectionView=None, zoomFixed=None, zoomLimits=(1.0, None))